## 데이터 가져오기

In [59]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


# 데이터프레임에서 특성과 레이블 나누기
X = pd.read_csv("./data/train_data_final.csv")  # 데이터 불러오기
y = pd.read_csv("./data/y.csv")  # 데이터 불러오기

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [61]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((32404, 59), (8101, 59), (32404, 1), (8101, 1))

## 미니 배치 학습

### RandomForestClassifier 모델을 사용하여 학습

In [13]:
# 미니 배치 생성 함수 정의
def create_mini_batches(X, y, batch_size):
    mini_batches = []
    data = np.array(X)
    labels = np.array(y)
    
    # 데이터와 레이블을 결합한 후 섞기
    combined = np.column_stack((data, labels))
    np.random.shuffle(combined)

    # 미니 배치 생성
    for i in range(0, len(combined), batch_size):
        mini_batch = combined[i:i + batch_size]
        mini_batches.append((mini_batch[:, :-1], mini_batch[:, -1]))  # 특성과 레이블 분리
        
    return mini_batches

# Random Forest 모델 정의
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 하이퍼파라미터 설정
batch_size = 32
epochs = 10

# 미니 배치 학습
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    mini_batches = create_mini_batches(X_train, y_train, batch_size)
    for X_mini_batch, y_mini_batch in mini_batches:
        model.fit(X_mini_batch, y_mini_batch)  # 각 미니 배치에 대해 학습 진행

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [14]:
# 최종 학습 후 전체 테스트 데이터에 대해 예측
y_pred = model.predict(X_test)


c:\lg_capstone\lg_capstone\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [29]:
# F1 스코어 평가
#y_test = y_test.values.flatten()
f1 = f1_score(y_test, y_pred, pos_label='Normal')
print(f'F1 Score: {f1:.6f}')

F1 Score: 0.968674


### XGBoost 모델을 사용하여 미니 배치 학습을 수행

In [60]:
y_train.shape

(32404, 1)

In [67]:
import numpy as np
from xgboost import XGBClassifier, plot_importance
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 정의 및 학습
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # y_train을 숫자로 인코딩

# XGBoost 모델 정의
model_xgb = XGBClassifier(eval_metric='mlogloss', reg_lambda=1, n_estimators=100, random_state=42)

# 하이퍼파라미터 설정
batch_size = 32
epochs = 10

# 미니 배치 학습
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    mini_batches = create_mini_batches(X_train, y_train_encoded, batch_size)  # y_mini_batch도 숫자로 변환
    for X_mini_batch, y_mini_batch in mini_batches:
        print(y_mini_batch)
        #model_xgb.fit(X_mini_batch, y_mini_batch.astype(int).ravel())  # 각 미니 배치에 대해 학습 진행

c:\lg_capstone\lg_capstone\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 0. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 1.]

### CatBoost 모델을 사용하여 미니 배치 학습을 수행

In [ ]:
from catboost import CatBoostClassifier

# CatBoost 모델 정의
model_cb = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0)

# 하이퍼파라미터 설정
batch_size = 32
epochs = 10

# 미니 배치 학습
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    mini_batches = create_mini_batches(X_train, y_train, batch_size)
    for X_mini_batch, y_mini_batch in mini_batches:
        model_cb.fit(X_mini_batch, y_mini_batch)  # 각 미니 배치에 대해 학습 진행

## 제출하기

In [74]:
test_data_final = pd.read_csv("data/test_data_final.csv")

In [75]:
y_pred_from_test = model.predict(test_data_final)

c:\lg_capstone\lg_capstone\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [80]:
type(y_pred_from_test)

numpy.ndarray

In [79]:
submission = pd.read_csv("data/submission.csv")

In [81]:
submission['target'] = y_pred_from_test

In [82]:
submission.to_csv("data/submission_final.csv", index=False)

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}



Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 110}
F1 Score: 0.6146
Confusion Matrix:
[[298 165]
 [197 280]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.60      0.64      0.62       463
         1.0       0.63      0.59      0.61       477

    accuracy                           0.61       940
   macro avg       0.62      0.62      0.61       940
weighted avg       0.62      0.61      0.61       940